Importar librerias:

In [4]:
import pandas as pd
import json as js
from pandas import json_normalize
import ast as ast
import gzip
import re
import numpy as np
import pyarrow

Ruta de archivos 

In [5]:
arc_json = r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\user_reviews.json.gz'
arc_json1 = r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\users_items.json.gz'
arc_json2 = r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\steam_games.json.gz'

Leer y procesar archivos gzip de user_reviews

In [6]:
lista_reviews = []
with gzip.open(arc_json, 'rt', encoding='utf-8') as files:
    for line in files.readlines():
        lista_reviews.append(ast.literal_eval(line))


df_reviews = pd.DataFrame(lista_reviews)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [7]:
df_reviews_copy = df_reviews.copy()

In [8]:
df_expande = df_reviews_copy.explode('reviews')

In [9]:
df_expande.reset_index(drop=True, inplace=True)

In [10]:
df_review_normalizade = json_normalize(df_expande['reviews'])

In [11]:
df_review = pd.concat([df_expande.drop(columns=['reviews']), df_review_normalizade], axis=1)

In [12]:
column_drop = ['funny','last_edited','helpful']
df_reviews = df_review.drop(columns= column_drop, errors = 'ignore')
df_reviews

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D


In [13]:
df_reviews.to_csv('df_review_clean.csv', index= False)

In [14]:
df_reviews.to_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_review_clean.parquet', index= False)

In [15]:
lista_items = []
with gzip.open(arc_json1, 'rt', encoding='utf-8') as files:
    for line in files.readlines():
        lista_items.append(ast.literal_eval(line))


df_items = pd.DataFrame(lista_items)
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [16]:
df_items_new = df_items.explode('items')

In [17]:
df_items_new.reset_index(drop=True, inplace=True)

In [18]:
df_items_normalizade = json_normalize(df_items_new['items'])

In [19]:
df_items_end = pd.concat([df_items_new.drop(columns=['items']), df_items_normalizade], axis=1)

In [20]:
df_items_end

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


In [21]:
column_drop = ['steam_id','user_url','item_name','playtime_2weeks']
df_items = df_items_end.drop(columns= column_drop, errors = 'ignore')
df_items

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


In [22]:
df_items.to_csv('df_items_clean.csv', index= False)

In [23]:
df_items.to_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_items_clean.parquet', index= False)

In [24]:
lista_games = []
with gzip.open(arc_json2, 'rt', encoding='utf-8') as files:
    for line in files.readlines():
        lista_games.append(js.loads(line))


df_games = pd.DataFrame(lista_games)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [25]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [26]:
df_games = df_games.drop(df_games.index[:88310])
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [27]:
df_array = df_games["app_name"] == df_games["title"]
df_array.info

<bound method Series.info of 88310      True
88311      True
88312      True
88313      True
88314     False
          ...  
120440     True
120441     True
120442     True
120443     True
120444    False
Length: 32135, dtype: bool>

In [28]:
Valores_iguales = df_array.value_counts().get(True,0)
print(Valores_iguales)
Valores_dif = df_array.value_counts().get(False,0)
print(Valores_dif)

29530
2605


In [29]:
nulos_titulos = df_games["title"].isnull()
cant_nulos_titulos = nulos_titulos.sum()
print(f"En la columna title hay una cantidad de {cant_nulos_titulos} valores nulos")

nulos_app = df_games["app_name"].isnull()
cant_nulos_app = nulos_app.sum()
print(f"En la columna app_name hay una cantidad de {cant_nulos_app} valores nulos")

nulos = cant_nulos_app + cant_nulos_titulos
filas_sin_nulos = Valores_dif - nulos
print(f"La cantidad de valores no nulos son {filas_sin_nulos}")

En la columna title hay una cantidad de 2050 valores nulos
En la columna app_name hay una cantidad de 2 valores nulos
La cantidad de valores no nulos son 553


In [30]:
files_nulls = df_games[df_games["app_name"].isnull()]
for i, filas in files_nulls.iterrows():
    print("Indice: {i}")
    print(filas)

Indice: {i}
publisher                                  NaN
genres                                     NaN
app_name                                   NaN
title                                      NaN
url             http://store.steampowered.com/
release_date                               NaN
tags                                       NaN
reviews_url                                NaN
specs                                      NaN
price                                    19.99
early_access                             False
id                                         NaN
developer                                  NaN
Name: 88384, dtype: object
Indice: {i}
publisher                                                     NaN
genres                                            [Action, Indie]
app_name                                                      NaN
title                                                         NaN
url                   http://store.steampowered.com/app/317160/_/
release_

In [31]:
df_games = df_games.drop('title', axis=1)
df_games

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [32]:
df_games['genres'].fillna(df_games['tags'], inplace=True)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_14848\1624767724.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games['genres'].fillna(df_games['tags'], inplace=True)


In [33]:
df_games['tags'].fillna(df_games['genres'], inplace=True)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_14848\2361507259.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games['tags'].fillna(df_games['genres'], inplace=True)


In [34]:
df_games

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,"[Action, Indie, Casual, Sports]",Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [35]:
df_games['developer'].fillna(df_games['publisher'], inplace=True)

C:\Users\EFRAIN LINARES\AppData\Local\Temp\ipykernel_14848\872335754.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games['developer'].fillna(df_games['publisher'], inplace=True)


In [36]:
columns_drop = ['url','publisher', 'reviews_url','specs','early_access']
df_games = df_games.drop(columns=columns_drop, errors='ignore')
df_games

,genres,app_name,release_date,tags,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
88314,"[Action, Indie, Casual, Sports]",Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN
...,...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870,"xropi,stev3ns"


In [37]:
# Convertir los datos de genres a cadenas
df_games['genres'] = df_games['genres'].astype(str)

# Funcion para eliminaro los Free to Play de genres

def elim_free_to_play (genres):
    try:
        list_genres = ast.literal_eval(genres)
        if "Free to Play" in list_genres:
            list_genres.remove('Free to Play')
        
        return ', '.join(list_genres)
    except(SyntaxError, ValueError):
        return None

df_games['genres'] = df_games['genres'].apply(elim_free_to_play)
df_games

,genres,app_name,release_date,tags,price,id,developer
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
88311,"Indie, RPG, Strategy",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980,Secret Level SRL
88312,"Casual, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290,Poolians.com
88313,"Action, Adventure, Casual",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
88314,"Action, Indie, Casual, Sports",Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN
...,...,...,...,...,...,...,...
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS"""
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada
120442,"Indie, Racing, Simulation",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich
120443,"Casual, Indie",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870,"xropi,stev3ns"


In [38]:
df_games['genres'] = df_games['genres'].str.split(',').str[0].str.strip()

print(df_games['genres'])

88310           Action
88311            Indie
88312           Casual
88313           Action
88314           Action
              ...     
120440          Casual
120441          Casual
120442           Indie
120443          Casual
120444    Early Access
Name: genres, Length: 32135, dtype: object


In [39]:
def price_conversion (price):
    if isinstance(price, str):
        reemplazos = {
            'Free to Play' : 'Free',
            'Play for Free' : 'Free',
            'Free to Try' : 'Free',
            'Play for Free!' : 'Free',
            'Free to Use' : 'Free',
            'Free Demo' : 'Free',
        }
        
        for old, new in reemplazos.items():
            price = price.replace(old, new)
        return price
    else:
        return price


def convertir_nan(value):
    if isinstance(value, str) and value != 'Free':
        return np.nan
    else:
        return value

df_games['price'] = df_games['price'].apply(price_conversion)

df_games['price'] = df_games['price'].apply(convertir_nan)
df_games

,genres,app_name,release_date,tags,price,id,developer
88310,Action,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
88311,Indie,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,643980,Secret Level SRL
88312,Casual,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free,670290,Poolians.com
88313,Action,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
88314,Action,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",2.99,773570,NaN
...,...,...,...,...,...,...,...
120440,Casual,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS"""
120441,Casual,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada
120442,Indie,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich
120443,Casual,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870,"xropi,stev3ns"


In [40]:
df_games1 = df_games.copy()

In [41]:
columna_elim = ['tags']
df_games1 = df_games1.drop(columns=columna_elim)
df_games1

,genres,app_name,release_date,price,id,developer
88310,Action,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,Indie,Ironbound,2018-01-04,NaN,643980,Secret Level SRL
88312,Casual,Real Pool 3D - Poolians,2017-07-24,Free,670290,Poolians.com
88313,Action,弹炸人2222,2017-12-07,0.99,767400,彼岸领域
88314,Action,Log Challenge,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
120440,Casual,Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,Casual,LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,Indie,Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
120443,Casual,EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [42]:
df_games1['release_date'] = pd.to_datetime(df_games1.release_date, errors='coerce')

In [43]:
df_games1['release_date'] = df_games1['release_date'].dt.year

In [44]:
df_games1

,genres,app_name,release_date,price,id,developer
88310,Action,Lost Summoner Kitty,2018.0,4.99,761140,Kotoshiro
88311,Indie,Ironbound,2018.0,NaN,643980,Secret Level SRL
88312,Casual,Real Pool 3D - Poolians,2017.0,Free,670290,Poolians.com
88313,Action,弹炸人2222,2017.0,0.99,767400,彼岸领域
88314,Action,Log Challenge,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
120440,Casual,Colony On Mars,2018.0,1.99,773640,"Nikita ""Ghost_RUS"""
120441,Casual,LOGistICAL: South Africa,2018.0,4.99,733530,Sacada
120442,Indie,Russian Roads,2018.0,1.99,610660,Laush Dmitriy Sergeevich
120443,Casual,EXIT 2 - Directions,2017.0,4.99,658870,"xropi,stev3ns"


In [45]:
df_games3 = df_games1.copy()

In [46]:
# Lambda para convertir a 0 si es string,sino mantiene el valor original
replace_string_with_zero = lambda x: 0 if isinstance(x, str) else x

# Aplico lambda a 'price'
df_games3['price'] = df_games3['price'].apply(replace_string_with_zero)

print(df_games3['price'])

88310     4.99
88311      NaN
88312     0.00
88313     0.99
88314     2.99
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 32135, dtype: float64


In [64]:
df_games3

,genres,app_name,release_date,price,id,developer
88310,Action,Lost Summoner Kitty,2018.0,4.99,761140,Kotoshiro
88311,Indie,Ironbound,2018.0,0.00,643980,Secret Level SRL
88312,Casual,Real Pool 3D - Poolians,2017.0,0.00,670290,Poolians.com
88313,Action,弹炸人2222,2017.0,0.99,767400,彼岸领域
88314,Action,Log Challenge,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
120440,Casual,Colony On Mars,2018.0,1.99,773640,"Nikita ""Ghost_RUS"""
120441,Casual,LOGistICAL: South Africa,2018.0,4.99,733530,Sacada
120442,Indie,Russian Roads,2018.0,1.99,610660,Laush Dmitriy Sergeevich
120443,Casual,EXIT 2 - Directions,2017.0,4.99,658870,"xropi,stev3ns"


In [45]:
df_games.to_csv("games_clean.csv", index=False)


In [46]:
df_games1.to_csv('df_games1_clean.csv', index=False)


In [65]:
df_games3.to_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_games3_clean.parquet', index=False)